In [ ]:
import torch
import numpy as np
from torch import nn
from functools import partial
from time import time
import json

## Data loading and processing

In [ ]:
adj_train = np.load("./ppi/adj_train.npz")
features = np.load("./ppi/feats.npy")

# The data for the (symmetric) adjacency matrix
data = adj_train["data"]
data = data.astype(int)
indices = adj_train["indices"]
indptr = adj_train["indptr"]
shape = adj_train["shape"]

# Load the labels
with open("./ppi/class_map.json", "r") as f:
    class_map = json.loads(f.read())
labels = np.empty((14755, 121))
for key, value in class_map.items():
    labels[int(key)] = value

# # This is a less memory-efficient method to get the sparse torch adjacency matrix, that also requires SciPy
# adj_matrix = csr_matrix((data, indices, indptr), shape=shape).toarray().astype(int)
# np.set_printoptions(threshold=sys.maxsize)
# adj_matrix = torch.from_numpy(adj_matrix).to_sparse()

# Change the SciPy csr format to torch format
torch_first_indices = []
for i in range(len(indptr)-1):
    torch_first_indices += [i for ind in indices[indptr[i]:indptr[i+1]]]
torch_first_indices = np.asarray(torch_first_indices)
torch_indices = np.stack((torch_first_indices, indices))

# The given shape for the ppi train set is much larger than the number of actual nodes in the set
num_nodes = len(np.unique(torch_indices))
shape_small = [num_nodes, num_nodes]

# Create the adjacency matrix
adj_matrix = torch.sparse_coo_tensor(indices=torch_indices, values=data, size=shape_small, dtype=torch.float)

# Calculate the node degrees
degree = [0.0 for i in range(num_nodes)]
for i in torch_indices.T:
    degree[i[0]] += 1
    if i[1] == i[0]:
        degree[i[0]] += 1
inverse_degree = np.reciprocal(np.asarray(degree))
# print(inverse_degree.shape)
# print(inverse_degree)

# Calculate the normalized adjacency matrix
norm_adj_data = inverse_degree[torch_indices[0]]*data.astype(float)

# norm_adj_matrix = torch.sparse_coo_tensor(indices=torch_indices, values=norm_adj_data.astype(float), size=shape_small,
#                                           dtype=torch.float64)

## Sampling

In [ ]:
def node_sampler(num_nodes, budget, p_nodes, p_edges, N, indices):
    return np.unique(np.random.choice(np.arange(num_nodes), size=budget, p=p_nodes, replace=True))

def edge_sampler(num_nodes, budget, p_nodes, p_edges, N, indices):
    p_edges = p_edges/p_edges.sum()
    edge_ids = np.unique(np.random.choice(np.arange(indices.shape[1]), size=budget, p=p_edges, replace=True))
    nodes_s = np.unique(np.take(indices, list(edge_ids)).flatten())
    return nodes_s

def custom_sampler(num_nodes, budget, p_nodes, p_edges, N, indices):
    nodes_s = np.unique(np.random.choice(np.arange(num_nodes), size=budget, p=p_nodes, replace=True))
    for i in range(1, N):
        condition = np.any(np.in1d(indices.flatten(), nodes_s).reshape(indices.shape), axis=0)
        expansion = np.unique(indices[:,condition].flatten())
        neighbours = np.setdiff1d(expansion, nodes_s)
        p_neigbours = p_nodes[neighbours]
        p_neigbours = p_neigbours/p_neigbours.sum()
        nodes_s = np.union1d(nodes_s, np.unique(np.random.choice(neighbours, size=budget, p=p_neigbours, replace=True)))
    return nodes_s

# Note that p_nodes_sampler is used as a parameter of sampling, while p_nodes represent the true probabilities
def sample_nodes(num_nodes, budget, p_nodes_sampler, p_nodes, p_edges, N, indices, data, features, labels, sampler):
    # Sample
    nodes_s = sampler(num_nodes, budget, p_nodes_sampler, p_edges, N, indices)
    num_nodes_s = len(nodes_s)
    
    # Connect the sampled nodes
    condition = np.all(np.in1d(indices.flatten(), nodes_s).reshape(indices.shape), axis=0)
    edges_s = indices[:,condition]
    data_s = data[condition]
    edge_indices = np.where(condition)[0]
    nodes_s = set(nodes_s)

    # Remove unconnected nodes (not connected to other nodes or themselves)
    len_before = len(nodes_s)
    nodes_s = nodes_s.intersection(set(np.unique(edges_s)))
    len_after = len(nodes_s)
    num_nodes_s -= len_before-len_after
    
    # If no nodes are connected, retry sampling
    if len(nodes_s) == 0:
        return sample_nodes(num_nodes, budget, p_nodes, p_edges, indices, data, features)
    
    orig2sub = {ind : i for i, ind in enumerate(nodes_s)}
    nodes_s_sub = {orig2sub[node] for node in nodes_s}
    edges_s_sub = np.vectorize(orig2sub.get)(edges_s)

    # Create the adjacency matrix of the sampled graph
    adj_matrix_s = torch.sparse_coo_tensor(indices=edges_s_sub, values=data_s, size=[num_nodes_s, num_nodes_s],
                                           dtype=torch.float)
    
    # Calculate the normalizing constants
    p_nodes_s = np.take(p_nodes, list(edges_s[0]))
    p_edges_s = np.take(p_edges, list(edge_indices))
    alpha = p_edges_s/p_nodes_s
    alpha_recip = 1/alpha
    alpha_matrix = torch.sparse_coo_tensor(indices=edges_s_sub, values=alpha_recip, size=[num_nodes_s, num_nodes_s],
                                           dtype=torch.float)
    
    # Select the relevant features, labels and loss normalizers
    features_s = torch.from_numpy(features[list(nodes_s)]).to(torch.float)
    labels_s = torch.from_numpy(labels[list(nodes_s)]).to(torch.float)
    norm_loss = torch.from_numpy(num_nodes*np.take(p_nodes, list(nodes_s))).to(torch.float).reshape(len(nodes_s),1)
    return adj_matrix_s * alpha_matrix, features_s, labels_s, norm_loss

In [ ]:
budget = 6000
N = 0

# Calculate the sampling probablities
p_nodes = [0.0 for i in range(shape_small[0])]
for i, ind in enumerate(torch_indices[1]):
    p_nodes[ind] += norm_adj_data[i]**2
p_nodes = np.asarray(p_nodes)
p_nodes = p_nodes/p_nodes.sum()

# Get the edge probabilities
# For the node sampler, the probability of an edge being sampled, is just to probability of both it's nodes being sampled
# Note that for an edge connecting a node to itself, the probability of sampling it is just the probability of sampling the node
self_loops = np.where(np.all(torch_indices == torch_indices[0,:], axis = 0)==True)
p_edges = np.take(p_nodes, torch_indices)
np.put(p_edges[1], self_loops, 1)
p_edges = p_edges.prod(0)

# p_edge_matrix = torch.sparse_coo_tensor(indices=torch_indices, values=p_edges, size=shape_small, dtype=torch.float64)
nodewise_sampler = partial(sample_nodes, num_nodes, budget, p_nodes, p_nodes, p_edges, N, torch_indices, norm_adj_data,
                           features, labels, node_sampler)
# nodewise_sampler()

In [ ]:
budget = 2000
N = 3
num_samples = 1000

# Sample using the node-wise probabilities to start, sample subgraphs and count node-occurances
# Note that p_nodes could have been re-used from the node-wise calculations
p_nodes = [0.0 for i in range(shape_small[0])]
for i, ind in enumerate(torch_indices[1]):
    p_nodes[ind] += norm_adj_data[i]**2
p_nodes = np.asarray(p_nodes)
p_nodes = p_nodes/p_nodes.sum()

# To make the algorithm more efficient, these samples could be reused for training
count_nodes = np.asarray([0.0 for i in range(shape_small[0])])
for i in range(num_samples):
    nodes_s = custom_sampler(num_nodes, budget, p_nodes, p_edges, N, torch_indices)
    for node in nodes_s:
        count_nodes[node] += 1
# Add a small amount to the counts if needed, to prevent division by zero errors down the line
if 0 in count_nodes:
    count_nodes += 0.1
p_nodes_approx = count_nodes/count_nodes.sum()

# Get the edge probabilities
self_loops = np.where(np.all(torch_indices == torch_indices[0,:], axis = 0)==True)
p_edges_approx = np.take(p_nodes, torch_indices)
np.put(p_edges_approx[1], self_loops, 1)
p_edges_approx = p_edges_approx.prod(0)

ladies_sampler = partial(sample_nodes, num_nodes, budget, p_nodes, p_nodes_approx, p_edges_approx, N, torch_indices,
                         norm_adj_data, features, labels, custom_sampler)
# ladies_sampler()

In [ ]:
budget = 4000
N = 0

# Calculate the edge probabilities based on the calculated degrees
p_edges = inverse_degree[torch_indices[0]] + inverse_degree[torch_indices[1]]
p_edges = p_edges/p_edges.sum()

# Calcualte the node probabilities using the edge probabilities
not_p_edges = 1 - p_edges
not_p_nodes = [1.0 for i in range(shape_small[0])]
for i, edge in enumerate(torch_indices.transpose()):
    not_p_nodes[edge[0]] *= not_p_edges[i]
not_p_nodes = np.asarray(not_p_nodes)
p_nodes = 1 - not_p_nodes

edgewise_sampler = partial(sample_nodes, num_nodes, budget, p_nodes, p_nodes, p_edges, N, torch_indices, norm_adj_data,
                           features, labels, edge_sampler)
# edgewise_sampler()

In [ ]:
class myGraphSAINT(nn.Module):
    def __init__(self, hidden_sizes, lr, p_dropout, sampler):
        super(myGraphSAINT, self).__init__()
        # The weights are initialized according to the default pytorch initialization for linear layers
        self.weights = torch.nn.ParameterList(
            nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty((hidden_sizes[i], hidden_sizes[i+1])), gain=1/np.sqrt(6.0)))
            for i in range(len(hidden_sizes)-1)
        )
        self.p_dropout = p_dropout
        self.sampler = sampler
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
    
    def forward(self):
        A, x, labels_s, norm_loss = self.sampleGraph()
        for W in self.weights:
            x = nn.functional.relu(A @ x @ W)
            x = torch.nn.Dropout(p=self.p_dropout)(x)
        x = nn.Sigmoid()(x)
        return x, labels_s, norm_loss
    
    def sampleGraph(self):
        return self.sampler()
    
    def train_step(self):
        self.optimizer.zero_grad()
        preds, labels_s, norm_loss = self()
        loss = torch.nn.BCEWithLogitsLoss(weight=norm_loss,reduction='sum')(preds, labels_s)
        loss.backward()
        self.optimizer.step()
        return loss.item()/preds.size(0)
    
    def fit(self, num_iterations):
        self.train()
        losses = []
        for i in range(num_iterations):
            losses.append(self.train_step())
        return losses

In [ ]:
n_its = 1000
hidden_size = 128

nodewise_model = myGraphSAINT([50, hidden_size, 121], 0.01, 0.0, nodewise_sampler)
edgewise_model = myGraphSAINT([50, hidden_size, 121], 0.01, 0.1, edgewise_sampler)
ladies_model = myGraphSAINT([50, hidden_size, 121], 0.01, 0.0, ladies_sampler)
t1 = time()
losses_nodewise = nodewise_model.fit(n_its)
t2 = time()
print("Node-Wise Training Time:", t2-t1)
losses_edgewise = edgewise_model.fit(n_its)
t3 = time()
print("Edge-Wise Training Time:", t3-t2)
losses_ladies = ladies_model.fit(n_its)
t4 = time()
print("Ladies Training Time:", t4-t3)

print()
print(np.asarray(losses_nodewise))
print()
print(np.asarray(losses_edgewise))
print()
print(np.asarray(losses_ladies))